# Unfinished

In [1]:
import pandas as pd
import os
import unicodedata
import re

# Library auxiliars

In [2]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def normalize_string(s):
    if s=='':
        return s
    s = strip_accents(s.strip()).lower()
    return re.sub('\s+', ' ', s) #remove multiple spaces

# Config files and folders
Path to input/output folders should be passed by parameter or should we use common ones?

Config files (csv):
- cod_to_proc = name of the databases to process
- info_proc = specifies the fields to preprocess and how to


In [3]:
configDir = os.path.join(os.getcwd(), '../config')
dictionariesDir = os.path.join(os.getcwd(), '../dicts')

databasesDir = os.path.join(os.getcwd(), '../example')

outputPath = os.path.join(databasesDir, '../instances')
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

In [4]:
# Dictionaries
countriesDict = pd.read_csv(f"{dictionariesDir}/dic_countries_iso2.csv")
countriesDict['Country.of.origin'] = countriesDict['Country.of.origin'].apply(lambda x: normalize_string(x))


genderDict = pd.read_csv(f"{dictionariesDir}/dic_gender.csv")
genderDict.Input = genderDict.Input.str.upper()
genderDict = dict(zip(genderDict.Input, genderDict.Gender))


locationsDict = pd.read_csv(f"{dictionariesDir}/dic_locations.csv")

In [5]:
# config files
parameters = pd.read_csv(f"{configDir}/parameters.csv")
# parameters = dict(zip(parametersDF.VAR, parametersDF.VALUE))
info_proc = pd.read_csv(f"{configDir}/info_proc.csv")
select_bases = pd.read_csv(f"{configDir}/select_bases.csv")

# Preprocess
+ The final script should allow to pass the names of the csv (databases) by command line or use a csv file named cod_to_proc.csv with the name of the databases to preprocess (to have compatibility with R's script)
+ the databases could be in its own directory (as in the example) and the name could be different (as in the example BA1_input instead of BA1)

In [6]:
# get databases names
databaseNames = pd.read_csv(f"{configDir}/cod_to_proc.csv", header=None)[0]

# for dbName in databaseNames: #TODO do the next for all databases
dbName = databaseNames[0]

# read database
db = pd.read_csv(f"{databasesDir}/{dbName}/{dbName}_input.csv")

db_info_proc = info_proc[info_proc.Cod == dbName]

print(f"original columns: {list(db.columns)}")
print(f"grab only: {list(db_info_proc.var_original)}")
print(f"change the name to: {list(db_info_proc.var_new)}")

# Only retain the fields from info_proc
db = db[db_info_proc.var_original]

# And change them as said in info_proc
db.columns = list(db_info_proc.var_new)

print(db_info_proc)
db

original columns: ['ID', 'Full.name', 'Country.of.origin', 'Age', 'Gender', 'Rol']
grab only: ['ID', 'Full.name', 'Country.of.origin', 'Gender', 'Age', 'Rol']
change the name to: ['Case.ID', 'Name_1', 'Nat', 'Sex', 'Age', 'Rol']
   Cod  var_new       var_original var_type  is.proc  to.join
0  BA1  Case.ID                 ID       ID        0        1
1  BA1   Name_1          Full.name     name        1        1
2  BA1      Nat  Country.of.origin      nat        1        1
3  BA1      Sex             Gender      sex        1        1
4  BA1      Age                Age      age        1        1
5  BA1      Rol                Rol     role        1        1


,Case.ID,Name_1,Nat,Sex,Age,Rol
0,BA1-01,Jorge Perez,Denmark,M,11.0,Missing
1,BA1-02,John McGregor,guinee equatoriale,M,NaN,Searching
2,BA1-03,Carla Prieto,zambia,F,34.0,Travelled with
3,BA1-04,Ariana Maradona,Guatemala,F,23.0,Informant
4,BA1-05,Oscar Alvarez,Jordan,M,44.0,Missing
5,BA1-06,Juan Perez,Mali,NaN,15.0,Missing
6,BA1-07,Maria Urcupiña,Peru,F,18.0,Missing
7,BA1-08,Stefania Roca,slovenia,NaN,NaN,Informant
8,BA1-09,Ramon Dekkers,shri lanka,M,22.0,Informant
9,BA1-10,Ringo Bonavena,tunesia,M,33.0,Informant


# Library

In [7]:
import re
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import unicodedata


In [8]:
# config files and dictionaries should be global variables 
#(or passed to every other function)

def processAge(age):
    return int(re.findall(r'\d+', age)[0])

def processAge_2(yob, toDate=date.today()):
    yob = datetime(int(yob),1,1) # year of birth - month: 1 - day: 1
    if 'fecha' in parameters.VAR:
        date_str = parameters[parameters.VAR == 'fecha'].VALUE.values[0]
        toDate = datetime.strptime(date_str, "%Y-%m-%d")
    return relativedelta(toDate, yob).years

# TODO def processAge_3

def processName(name):
    return normalize_string(name)

def processSex(sex):
    sex = sex.strip().upper()
    return genderDict[sex]
    
def processDate(date_str, dateFormat = "%m/%d/%Y"):
    return datetime.strptime(date_str, dateFormat)

# TODO def processLocation(location):

def processNationality(nationality):
    nationality = normalize_string(nationality)
    if nationality in countriesDict['Country.of.origin']:
        print('found')
        nationality = countriesDict[countriesDict['Country.of.origin']==nationality].Processed.values[0]
    return nationality #best effort, otherwise we do nothing
    


In [9]:
processTypeDict = {'age': processAge,
    'age_2':processAge_2,
#     'age_3':processAge_3,
    'sex': processSex,
    'nat': processNationality,
    'name': processName,
    'date': processDate,
#     'loc': processLoc,
#     'adress': processAdress,
#     'phone': processPhone,
    'role': processName
}

In [10]:
processNationality('africa')
nationality='africa'
countriesDict['Country.of.origin']

withIndex = countriesDict.set_index('Country.of.origin')
withIndex.to_dict()

{'Processed': {'afghanistan': 'AFGHANISTAN',
  'afganistan': 'AFGHANISTAN',
  'africa': 'AFRICA',
  'albania': 'ALBANIA',
  'algeria': 'ALGERIA',
  'algerie': 'ALGERIA',
  'andorra': 'ANDORRA',
  'angola': 'ANGOLA',
  'palestine': 'AREA UNDER PALESTINIAN AUTHORITY',
  'palestina': 'AREA UNDER PALESTINIAN AUTHORITY',
  'argentina': 'ARGENTINA',
  'armenia': 'ARMENIA',
  'asia': 'ASIA',
  'austria': 'AUSTRIA',
  'azerbaijan': 'AZERBAIJAN',
  'bangladesh': 'BANGLADESH',
  'noakhali bangladesh': 'BANGLADESH',
  'belarus': 'BELARUS',
  'belgium': 'BELGIUM',
  'benin': 'BENIN',
  'bhutan': 'BHUTAN',
  'bolivia': 'BOLIVIA',
  'bosnia and herzegovina': 'BOSNIA AND HERZEGOVINA',
  'bosnia': 'BOSNIA AND HERZEGOVINA',
  'brazil': 'BRAZIL',
  'bulgaria': 'BULGARIA',
  'burkina faso': 'BURKINA FASO',
  'burkina f.': 'BURKINA FASO',
  'burundi': 'BURUNDI',
  'cameroon': 'CAMEROON',
  'camerun': 'CAMEROON',
  'cameroun': 'CAMEROON',
  'cape verde': 'CAPE VERDE',
  'cabo verde': 'CAPE VERDE',
  'centr